Disclaimer : Ini merupakan hasil atau materi pembelajaran saya, selama saya belajar dari Pacmann

#Foundation of Statistic -  Model Evaluation 1

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**Outline**

1. Topic Review
2. Case 1 - Sales Prediction
3. Case 2 - Medical Charge Prediction

- We need to import library and function that need to run the code in this notebook to:
  - load data
  - doing simulation
  - drawing graph and other visualization
  - performing cross validation using statsmodel

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as pyplot
import seaborn as sns

from matplotlib.gridspec import GridSpec

import statsmodels.api as sm
import statsmodels.formula.api as smf

from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.metrics import r2_score

from sklearn.model_selection import cross_validate, cross_val_score, KFold


In [ ]:
def print_coef_std_err(results):
    """
    Function to combine estimated coefficients and standard error in one DataFrame
    :param results: <statsmodels RegressionResultsWrapper> OLS regression results from statsmodel
    :return df: <pandas DataFrame>  combined estimated coefficient and standard error of model estimate
    """
    coef = results.params
    std_err = results.bse

    df = pd.DataFrame(data = np.transpose([coef, std_err]),
                      index = coef.index,
                      columns=["coef","std err"])
    return df

class StatsmodelsRegressor(BaseEstimator, RegressorMixin):
    """ A universal sklearn-style wrapper for statsmodels regressors """

    def __init__(self, sm_class, sm_formula):
        self.sm_class = sm_class
        self.sm_formula = sm_formula
        self.model = None
        self.result = None

    def fit(self, data, dummy):
        self.model = self.sm_class(self.sm_formula, data)
        self.result = self.model.fit()

    def predict(self,X):
        return self.result.predict(X)

def kfold_split(data, n_fold):
    """
    Function to combine estimated coefficients and standard error in one DataFrame
    :param results: <statsmodels RegressionResultsWrapper> OLS regression results from statsmodel
    :param n_fold: <int> Number of fold in k-fold CV
    :return df: <pandas DataFrame>  combined estimated coefficient and standard error of model estimate
    """
    kfold = KFold(n_splits = n_fold,
                  shuffle = True,
                  random_state=123)
    fold_train = {}
    fold_test = {}

    for i, (train, test) in enumerate(kfold.split(data)):
        print(f"fold {i+1}, train data rows: {len(train)}, test data rows: {len(test)}")
        fold_train[i] = data.iloc[train].copy()
        fold_test[i] = data.iloc[test].copy()

    return (fold_train, fold_test)

In [ ]:
    """
    Function to combine estimated coefficients and standard error in one DataFrame
    :param results: <statsmodels RegressionResultsWrapper> OLS regression results from statsmodel
    :return df: <pandas DataFrame>  combined estimated coefficient and standard error of model estimate
    """

#def print_coef_std_err(results):
#  coef = results.params
#  std_err = results.bse

#  df = pd.DataFrame(data=np.transpose([coef, std_err]),
#                      index=results.params.index,
#                      columns=['coef', 'std_err'])
#  return df

'\nFunction to combine estimated coefficients and standard error in one DataFrame\n:param results: <statsmodels RegressionResultsWrapper> OLS regression results from statsmodel\n:return df: <pandas DataFrame>  combined estimated coefficient and standard error of model estimate\n'

In [ ]:
""" A universal sklearn-style wrapper for statsmodels regressors """

#class StatsmodelsRegressor(BaseEstimator, RegressorMixin):
#  def __init__(self, sm_class, sm_formula):
#    self.sm_class = sm_class
#    self.sm_formula = sm_formula
#    self.model = None
#    self.result = None

#  def fit(self, data, dummy):
#    self.model = self.sm_class(self.sm_formula, data=data)
#    self.result = self.model.fit()

#  def predict(self, data):
#    return self.result.predict(X)

' A universal sklearn-style wrapper for statsmodels regressors '

In [ ]:
    """
    Function to combine estimated coefficients and standard error in one DataFrame
    :param results: <statsmodels RegressionResultsWrapper> OLS regression results from statsmodel
    :param n_fold: <int> Number of fold in k-fold CV
    :return df: <pandas DataFrame>  combined estimated coefficient and standard error of model estimate
    """
#def KFold_split(data, n_fold):
#  Kfold = KFold(n_splits=n_fold,
#                shuffle=True,
#                random_state=123)

#  fold_train = {}
#  fold_test = {}

#  for i, (train, test) in enumerate(kfold.split(data)):
#        print(f"fold {i+1}, train data rows : {len(train)}, test data rows : {len(test)}")
#       fold_train[i] = data.iloc[train].copy()
#        fold_test[i] = data.iloc[test].copy()

#  return (fold_train, fold_test)

'\nFunction to combine estimated coefficients and standard error in one DataFrame\n:param results: <statsmodels RegressionResultsWrapper> OLS regression results from statsmodel\n:param n_fold: <int> Number of fold in k-fold CV\n:return df: <pandas DataFrame>  combined estimated coefficient and standard error of model estimate\n'

## **Topic Review**

### Residual Standard Deviation and Explained Variance
- Residual standard deviation values are depend on the scale of outcome variable, so it's hard to tell that whether the value is good or not under the fitted model.
- R-squared is a metric to tell how much variance of outcome variable are explained in model. since the value is proportion, the closer the R-squared to 1, the better the model is.
$$R^2 = \frac{\sum_{i=1}^n(y_i-\hat{y}_i)^2}{\sum_{i=1}^n(y_i-\bar{y})^2}$$


  - There’s no hard-and-fast rule for how large R-square as it varies depending on the application. For instance, in macroeconomics, the regression lines commonly have R-square greater than 90%, while in medical research and marketing surveys, an R-square of 30% or less may signify a significant breakthrough.$^{[1]}$
  - R-squared can be used as metric evaluation of regression model

### Model Evaluation

- The goal of model evaluation is to make sure our fitted regression model are perform well in both observed data and unseen data
  - Cross Validation is one of technique that help us generalized the evaluation of model, by hold out some of our observation data as the proxy for future data.
  - Most well-known method to separate the data is K-Fold Cross Validation.
  - K-Fold Cross Validation devide our data to k partition, and fitting the model k times and computed each performance
  - The score in cross validation usually computed as the average of performance across k models





  - Initially, a dataset with n observations is randomly divided into five groups with no overlap.
  - Each of the five subsets takes turns acting as the training set (it may called as validation set too in other reference) (colored in beige), while the remaining subsets act as the training set (colored in blue).
  - The resulting performance is calculated for each of the five models, and the test error is estimated by averaging these performances, for example using R-squared.

$$\text{CV}_{(k)} = \frac{1}{k}\sum_{i=1}^k R^2_i$$

## **Case 1 - Sales Prediction**

Sales team wants to optimize their advertising budget. They have collected previous data about avertising budget in each TV, Radio and Newspaper as marketing channels and each overall sales revenue in Millions Dollar.

***They want to know***

1. Which channels that contribute positively to overall sales.
2. If they have total advertising budget`170$`, and they have sales target to achieve 25 millions dollar revenue. How to distribute the total budget to each channels?

Help team sales to answer the questions and give recommendation to sales team utilizing regression model

### Load Data

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Data Science /Dataset/Advertising.csv')
df.head()

,TV,Radio,Newspaper,Sales
0,230.1,37.8,69.2,22.1
1,44.5,39.3,45.1,10.4
2,17.2,45.9,69.3,9.3
3,151.5,41.3,58.5,18.5
4,180.8,10.8,58.4,12.9


In [ ]:
df.isna().sum()

,0
TV,0
Radio,0
Newspaper,0
Sales,0


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   TV         200 non-null    float64
 1   Radio      200 non-null    float64
 2   Newspaper  200 non-null    float64
 3   Sales      200 non-null    float64
dtypes: float64(4)
memory usage: 6.4 KB


In [ ]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
TV,200.0,147.0425,85.854236,0.7,74.375,149.75,218.825,296.4
Radio,200.0,23.2640,14.846809,0.0,9.975,22.90,36.525,49.6
Newspaper,200.0,30.5540,21.778621,0.3,12.750,25.75,45.100,114.0
Sales,200.0,14.0225,5.217457,1.6,10.375,12.90,17.400,27.0


#### Define the model to fit for the data

In [ ]:
# Create a class model
ols_all_media = StatsmodelsRegressor(smf.ols, "Sales ~ TV + Radio + Newspaper")

#### Evaluate the model using K-fold cross validation

In [ ]:
# Create k-fold splitter object
kfold = KFold(n_splits =5,shuffle = True, random_state =123)

In [ ]:
#### Calculate performance for each model
scores_ols_all_media = cross_val_score(estimator = ols_all_media,
                                       X = df,
                                       y = df["Sales"],
                                       scoring="r2",
                                       cv=kfold)

scores_ols_all_media = pd.DataFrame(data = scores_ols_all_media,
                                    columns = ["test_rsquared"])

scores_ols_all_media["folds"] = [f"Folds {i+1}" for i in range(5)]
scores_ols_all_media

,test_rsquared,folds
0,0.928861,Folds 1
1,0.846030,Folds 2
2,0.854402,Folds 3
3,0.885496,Folds 4
4,0.913986,Folds 5


In [ ]:
scores_ols_all_media["test_rsquared"].mean()

0.8857548804553106

The model use all media have good fit, it can explain 88.57% variance of sales. Let's build the model using whole data to infer the estimated coefficient from the model, and gain the insight from business prespective

### Fit Linear Regression - Using All Media Channel

We can obtain the relationship each channel to sales by fitting linear regression model

In [ ]:
# Create OLS model object
model = smf.ols("Sales ~ TV + Radio + Newspaper",  df)

# Fit the model
results = model.fit()
results.summary()


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  Sales   R-squared:                       0.897
Model:                            OLS   Adj. R-squared:                  0.896
Method:                 Least Squares   F-statistic:                     570.3
Date:                Tue, 25 Feb 2025   Prob (F-statistic):           1.58e-96
Time:                        05:58:01   Log-Likelihood:                -386.18
No. Observations:                 200   AIC:                             780.4
Df Residuals:                     196   BIC:                             793.6
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      2.9389      0.312      9.422      0.000       2.324       3.554
TV             0.0458      0.001     32.809      0.000       0.043       0.049
Radio          0.1885      0.009     21.893      0.000       0.172       0.206
Newspaper     -0.0010      0.006     -0.177      0.860      -0.013       0.011
==============================================================================
Omnibus:                       60.414   Durbin-Watson:                   2.084
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              151.241
Skew:                          -1.327   Prob(JB):                     1.44e-33
Kurtosis:                       6.332   Cond. No.                         454.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [ ]:
# Extract the results (Coefficient and Standard Error) to DataFrame
results_advert = print_coef_std_err(results)
results_advert

,coef,std err
Intercept,2.938889,0.311908
TV,0.045765,0.001395
Radio,0.188530,0.008611
Newspaper,-0.001037,0.005871


### Insight 1

- The estimated coefficient of Radio and TV channel is positive, it means the higher advertising cost in Radio or TV channel, also expect higher sales revenue.
- The coefficient in Newspaper is negative, although the coefficient is small, it means the higher advertising cost in Newspaper, expect lower sales revenue.

> 1. Which channels that contribute positively to overall sales.

Regarding the first question, we recommend to use Radio and TV channel to deliver the advertising as they have positive contribution to sales

> 2. If they have budget advertising 170 Millions dollar, and they have sales target to achieve 25. How to distribute 170 to each marketing channels?

Regarding the second question, we can use the estimated coefficient as the weight of channels. To know budget distribution in each channel, we need to do calculation based on the the regression equation and the budget constrained.

The model shows the relationship in this equation between sales revenue, radio and tv
 <div style="padding-left: 30px;">
        <br>
  $\text{Sales} = 2.9 + 0.19\text{Radio} + 0.05\text{TV} + 0.19\text{Radio} - 0.001\text{Newspaper} $
        <br>
  </div>

or we can write
  <div style="padding-left: 30px;">
        <br>
  $\text{Target Sales} = \text{intercept} + (\text{w_radio}\times\text{Radio Budget}) + (\text{w_tv}\times\text{TV Budget}) + (\text{w_newspaper}\times\text{Newspaper Budget})$
        <br>
  </div>
  


- We know expected target sales is 25, however we dont expect our sales to decrease by allocating budget to newspaper channel, so we set 0 budget to newspaper
- By the regression equation we can write
    <div style="padding-left: 30px;">
        <br>
            $2.9 + (0.19\times\text{Radio Budget}) + (0.05\times\text{TV Budget}) = \text{25} \text{ ........ (1)}$
        <br>
    </div>
    
    
- We have maximum total budget 170 Millions dollar, we can write,
<div style="padding-left: 190px;">
        <br>
$\text{Radio Budget} + \text{TV Budget} = \text{Total Budget}$,
        <br>
</div>
 <br>
 or
 <br>
    <div style="padding-left: 190px;">
        <br>
            $\text{Radio Budget} + \text{TV Budget} = 170 \text{ ........ (2)}$
        <br>
    </div>


- We need to calculate the value of each variables Radio and TV budget constrained by regression equation and possible total budget.
- We need to do simple calculation by subtitute equation (2) to equation (1)

$$2.9 + (0.19\times\text{Radio Budget}) + (0.05\times(170 - \text{Radio Budget})) = \text{25}$$

$$2.9 + (0.19\times\text{Radio Budget}) + (0.05\times170) - (0.05\times\text{Radio Budget})) = \text{25}$$

$$\text{Radio Budget} =\frac{25 - 2.9 - (0.05\times170)}{0.19-0.05}$$

For simplicity we can calculate the radio budget using this equation

$$\text{Radio Budget} = \frac{\text{Target Sales} - \text{Intercept} - (\text{w_tv}\times\text{Total Budget})}{\text{w_radio}-\text{w_tv}}$$

In [ ]:
intercept = results_advert["coef"]["Intercept"]
w_radio = results_advert["coef"]["Radio"]
w_tv = results_advert["coef"]["TV"]

total_buget = 170
target_sales = 25


In [ ]:
radio_budget = (target_sales - intercept - (w_tv*total_buget))/(w_radio-w_tv)
radio_budget

100.03210692337616

In [ ]:
tv_budget = total_buget - radio_budget
tv_budget

69.96789307662384

### Insight 2


> 1. If they have budget advertising 170 Millions dollar, and they have sales target to achieve 25. How to distribute 170 to each marketing channels?

Regarding the second question, if Sales Team expect 25 millions dollar sales revenue with total budget 170 millions dollar, Sales Team can use `69.46 millions dollar for tv` and `100.54 millions dollar for radio`